In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import VotingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos1.csv')
elos_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos2.csv')
elos_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos3.csv')
elos_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos4.csv')
features_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features1.csv')
features_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features3.csv')
features_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features2.csv')
features_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features4.csv')

In [4]:
X_train_mean_1, X_test_mean_1, Y_train_mean_1, Y_test_mean_1 =ms.train_test_split(features_1, elos_1['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_1, X_test_diff_1, Y_train_diff_1, Y_test_diff_1 =ms.train_test_split(features_1, elos_1['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_2, X_test_mean_2, Y_train_mean_2, Y_test_mean_2 =ms.train_test_split(features_2, elos_2['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_2, X_test_diff_2, Y_train_diff_2, Y_test_diff_2 =ms.train_test_split(features_2, elos_2['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_3, X_test_mean_3, Y_train_mean_3, Y_test_mean_3 =ms.train_test_split(features_3, elos_3['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_3, X_test_diff_3, Y_train_diff_3, Y_test_diff_3 =ms.train_test_split(features_3, elos_3['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_4, X_test_mean_4, Y_train_mean_4, Y_test_mean_4 =ms.train_test_split(features_4, elos_4['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_4, X_test_diff_4, Y_train_diff_4, Y_test_diff_4 =ms.train_test_split(features_4, elos_4['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_diff=VotingRegressor(estimators=[
                        ('rf_vt',RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
                      max_features=1, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=5, min_samples_split=25,
                      min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                      oob_score=True, random_state=4, verbose=0,
                      warm_start=False)),
    
                             ('svr_vt', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
                                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)),
                             ('lr_vt',LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))],
                           
                n_jobs=-1,weights=None)

In [6]:
model_mean=VotingRegressor(estimators=[
                        ('rf_vt',RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
                      max_features=2, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
                      oob_score=False, random_state=4, verbose=0,
                      warm_start=False)),
    
                             ('svr_vt', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
                                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)),
                             ('lr_vt',LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))],
                           
                 n_jobs=-1,weights=None)

In [7]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [8]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для 1 кластера

In [9]:
start1 = time.time()

voting1=model_mean.fit(X_train_mean_1,Y_train_mean_1)
voting_pred1=voting1.predict(X_test_mean_1)
scores1=predictions_report(Y_test_mean_1,voting_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 105.642
RMSE = 133.158
Correlation coefficient = 0.268
20.194 c


In [10]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [11]:
start2 = time.time()

voting2=model_diff.fit(X_train_diff_1,Y_train_diff_1)
voting_pred2=voting2.predict(X_test_mean_1)
scores2=predictions_report(Y_test_diff_1,voting_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 192.155
RMSE = 231.969
Correlation coefficient = 0.526
11.289 c


In [12]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [13]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [14]:
white_elos_1 = voting_pred1 + voting_pred2/2
black_elos_1 = voting_pred1-voting_pred2/2
pred_elos1=list(zip(white_elos_1,black_elos_1))
print(len(pred_elos1))
pred_elos1[:5]

1279


[(1884.1421331097956, 1805.3304525266938),
 (1901.706336414382, 1836.6109470509718),
 (1907.1950171294072, 1823.9845252905677),
 (1840.409152044084, 1773.8336081435602),
 (1827.4651552060097, 1881.5073790777894)]

In [15]:
find_abs_error(pred_elos1,elos_1),find_mean_error(pred_elos1,elos_1),find_root_mean_squared_error(pred_elos1,elos_1)

(399518.698, 156.184, 194.666)

In [16]:
inner_mae_1=find_mean_error(pred_elos1,elos_1)

In [17]:
errors =errors.append(pd.Series(['1 кластер','Voting Regressor',
                  find_abs_error(pred_elos1,elos_1),
                  find_mean_error(pred_elos1,elos_1),
                   find_root_mean_squared_error(pred_elos1,elos_1) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.93600,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.28400,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.55900,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.60100,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.84500,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.03200,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.14700,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.79500,122.90600
8,4 кластер,Random Forest,3.539965e+05,101.49000,125.63000
9,Общая ошибка,Random Forest,1.114983e+06,259.36600,334.37800


In [18]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для 2 кластера

In [19]:
start3 = time.time()

voting3=model_mean.fit(X_train_mean_2,Y_train_mean_2)
voting_pred3=voting3.predict(X_test_mean_2)
scores3=predictions_report(Y_test_mean_2,voting_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 77.289
RMSE = 95.490
Correlation coefficient = 0.172
42.728 c


In [20]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [21]:
start4 = time.time()

voting4=model_diff.fit(X_train_diff_2,Y_train_diff_2)
voting_pred4=voting4.predict(X_test_mean_2)
scores4=predictions_report(Y_test_diff_2,voting_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 115.149
RMSE = 142.457
Correlation coefficient = 0.480
30.704 c


In [22]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [23]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [24]:
white_elos_2 = voting_pred3 + voting_pred3/2
black_elos_2 = voting_pred3-voting_pred4/2
pred_elos2=list(zip(white_elos_2,black_elos_2))
print(len(pred_elos2))
pred_elos2[:5]

2466


[(3726.166952328159, 2452.286947824966),
 (3766.7524849224974, 2513.108645185877),
 (3718.1944897336525, 2449.479277786929),
 (3743.1392550767587, 2476.8065161184),
 (3730.5235163306, 2524.5743039815015)]

In [25]:
find_abs_error(pred_elos2,elos_2),find_mean_error(pred_elos2,elos_2),find_root_mean_squared_error(pred_elos2,elos_2)

(3317708.983, 672.69, 886.073)

In [26]:
inner_mae_2=find_mean_error(pred_elos2,elos_2)

In [27]:
errors =errors.append(pd.Series(['2 кластер','Voting Regressor',
                        find_abs_error(pred_elos2,elos_2),
                        find_mean_error(pred_elos2,elos_2),
                        find_root_mean_squared_error(pred_elos2,elos_2)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.93600,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.28400,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.55900,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.60100,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.84500,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.03200,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.14700,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.79500,122.90600
8,4 кластер,Random Forest,3.539965e+05,101.49000,125.63000
9,Общая ошибка,Random Forest,1.114983e+06,259.36600,334.37800


### Подбор модели для 3 кластера

In [28]:
start5 = time.time()

voting5=model_mean.fit(X_train_mean_3,Y_train_mean_3)
voting_pred5=voting5.predict(X_test_mean_3)
scores5=predictions_report(Y_test_mean_3,voting_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 82.325
RMSE = 96.470
Correlation coefficient = 0.156
28.97 c


In [29]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [30]:
start6 = time.time()

voting6=model_diff.fit(X_train_diff_3,Y_train_diff_3)
voting_pred6=voting6.predict(X_test_mean_3)
scores6=predictions_report(Y_test_diff_3,voting_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 105.971
RMSE = 137.389
Correlation coefficient = 0.357
17.088 c


In [31]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,3.101891e-01,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,5.649173e-01,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,2.175725e-01,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,5.015886e-01,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,2.378129e-01,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,3.901095e-01,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,2.286831e-01,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,4.048796e-01,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,2.695258e-01,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,5.273697e-01,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [32]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [33]:
white_elos_3 = voting_pred5 + voting_pred6/2
black_elos_3 = voting_pred5-voting_pred6/2
pred_elos3=list(zip(white_elos_3,black_elos_3))
print(len(pred_elos3))
pred_elos3[:5]

1789


[(2279.1276986643093, 2098.5186095526747),
 (2313.241074814106, 2085.2437384395657),
 (2320.81753640746, 2111.088458013683),
 (2325.8274857105066, 2104.968303779831),
 (2306.5401111959036, 2065.1896605099582)]

In [34]:
find_abs_error(pred_elos3,elos_3),find_mean_error(pred_elos3,elos_3),find_root_mean_squared_error(pred_elos3,elos_3)

(356251.702, 99.567, 122.565)

In [35]:
inner_mae_3=find_mean_error(pred_elos3,elos_3)

In [36]:
errors =errors.append(pd.Series(['3 кластер','Voting Regressor',
                                 find_abs_error(pred_elos3,elos_3),
                                 find_mean_error(pred_elos3,elos_3),
                                 find_root_mean_squared_error(pred_elos3,elos_3)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.93600,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.28400,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.55900,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.60100,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.84500,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.03200,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.14700,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.79500,122.90600
8,4 кластер,Random Forest,3.539965e+05,101.49000,125.63000
9,Общая ошибка,Random Forest,1.114983e+06,259.36600,334.37800


### Подбор модели для 4 кластера

In [37]:
start7 = time.time()

voting7=model_mean.fit(X_train_mean_4,Y_train_mean_4)
voting_pred7=voting7.predict(X_test_mean_4)
scores7=predictions_report(Y_test_mean_4,voting_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

MAE = 82.197
RMSE = 96.484
Correlation coefficient = 0.115
25.237 c


In [38]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [39]:
start8 = time.time()

voting8=model_diff.fit(X_train_diff_4,Y_train_diff_4)
voting_pred8=voting8.predict(X_test_mean_4)
scores8=predictions_report(Y_test_diff_4,voting_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

MAE = 107.381
RMSE = 188.127
Correlation coefficient = 0.067
16.692 c


In [40]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.980777,129.002684,0.310189,0.026,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",173.676816,218.061005,0.564917,0.032,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",77.387126,95.797568,0.217573,0.040,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",108.608625,135.853982,0.501589,0.039,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.616755,94.974990,0.237813,0.040,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",102.869006,132.827010,0.390109,0.028,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",80.319818,94.829941,0.228683,0.028,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",103.741893,134.630404,0.404880,0.037,Обучение на разнице в ЭЛО по 4 кластеру
8,"RandomForestRegressor(bootstrap=False, criteri...",106.924601,132.042952,0.269526,15.508,Обучение на среднем ЭЛО по 1 кластеру
9,"RandomForestRegressor(bootstrap=True, criterio...",188.770360,229.315473,0.527370,7.519,Обучение на разнице ЭЛО по 1 кластеру


In [41]:
white_elos_4 = voting_pred7 + voting_pred8/2
black_elos_4 = voting_pred7-voting_pred8/2
pred_elos4=list(zip(white_elos_4,black_elos_4))
print(len(pred_elos4))
pred_elos4[:5]

1744


[(2136.0384278584047, 2309.3847179533527),
 (2121.5235935950254, 2331.0718930831367),
 (2126.1943937111046, 2314.2522870587873),
 (2088.4119880191342, 2332.8783851407757),
 (2116.226257714642, 2322.6370480806618)]

In [42]:
find_abs_error(pred_elos4,elos_4),find_mean_error(pred_elos4,elos_4),find_root_mean_squared_error(pred_elos4,elos_4)

(357349.805, 102.451, 140.178)

In [43]:
inner_mae_4=find_mean_error(pred_elos4,elos_4)

In [44]:
errors =errors.append(pd.Series(['4 кластер','Voting Regressor',find_abs_error(pred_elos4,elos_4),
                                 find_mean_error(pred_elos4,elos_4),
                                 find_root_mean_squared_error(pred_elos4,elos_4)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.93600,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.28400,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.55900,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.60100,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.84500,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.03200,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.14700,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.79500,122.90600
8,4 кластер,Random Forest,3.539965e+05,101.49000,125.63000
9,Общая ошибка,Random Forest,1.114983e+06,259.36600,334.37800


In [45]:
voting_mae=(inner_mae_4+inner_mae_3+inner_mae_2+inner_mae_1)/4
voting_mae

257.723

In [46]:
voting_abs=(find_abs_error(pred_elos1,elos_1)+
        find_abs_error(pred_elos2,elos_2)+
        find_abs_error(pred_elos3,elos_3)+
        find_abs_error(pred_elos4,elos_4))/4
voting_abs

1107707.297

In [47]:
voting_rmse=(find_root_mean_squared_error(pred_elos1,elos_1)+
         find_root_mean_squared_error(pred_elos2,elos_2)+
         find_root_mean_squared_error(pred_elos3,elos_3)+
         find_root_mean_squared_error(pred_elos4,elos_4))/4
voting_rmse

335.8705

In [48]:
errors = errors.append(pd.Series(['Общая ошибка','Voting Regressor',
                  voting_abs,
                  voting_mae,
                 voting_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,1.150193e+06,134.93600,168.90800
1,2 кластер,LinearRegression,1.566282e+06,95.28400,117.43700
2,3 кластер,LinearRegression,1.127526e+06,94.55900,115.89300
3,4 кластер,LinearRegression,1.099642e+06,94.60100,116.29500
4,Общая ошибка,LinearRegression,1.235911e+06,104.84500,129.63325
5,1 кластер,Random Forest,4.042465e+05,158.03200,195.80200
6,2 кластер,Random Forest,3.344622e+06,678.14700,893.17400
7,3 кластер,Random Forest,3.570672e+05,99.79500,122.90600
8,4 кластер,Random Forest,3.539965e+05,101.49000,125.63000
9,Общая ошибка,Random Forest,1.114983e+06,259.36600,334.37800
